# FastSLAM2.0

In [1]:
import sys
sys.path.append('../scripts/')
from kf import * # this also import Mcl class
%matplotlib widget

In [2]:
class FastSlam2Agent(EstimationAgent): #EstimationAgent is in mcl.py, which is Agent+Estimator.  
    def __init__(self, time_interval, nu, omega, estimator):
        super().__init__(time_interval, nu, omega, estimator)
        
    def decision(self, observation=None): 
        self.estimator.motion_update(self.prev_nu, self.prev_omega, self.time_interval, observation) #add an observation.
        self.prev_nu, self.prev_omega = self.nu, self.omega
        self.estimator.observation_update(observation)
        return self.nu, self.omega

In [ ]:
class FastSlam2(Mcl):
    def __init__(self, init_pose, num_particle, num_landmark, motion_noise_stds={"nn":0.19, "no":0.001, "on":0.13, "oo":0.2},                  
                 distance_dev_rate=0.14, direction_dev=0.05):
        super().__init__(None, init_pose, num_particle, motion_noise_stds, distance_dev_rate, direction_dev) # let envmap None.
        self.particles = [MapParticle(init_pose, 1.0/num_particle, num_landmark) for i in range(num_particle)]
        self.ml = self.particles[0] # temporally
        self.motion_noise_stds = motion_noise_stds # add to use in motion_update2()
        
    def draw(self, ax, elems): #same with FastSlam
        super().draw(ax, elems)
        self.ml.map.draw(ax, elems)
        
    def observation_update(self, observation): #same with FastSlam
        for p in self.particles:
            p.observation_update(observation, self.distance_dev_rate, self.direction_dev) #MapParticle's function
        self.set_ml() # defined in Mcl.
        self.resampling()
        
    def motion_update(self, nu, omega, time, observation):
        not_first_obs = []
        for obs in observation:
            if self.particles[0].map.landmarks[obs[1]].cov is not None: # check first particle's map covariance
                not_first_obs.append(obs)
                
        if len(not_first_obs) > 0:
            for p in self.particles:
                p.motion_update2(nu, omega, time, \
                                 self.motion_noise_stds, not_first_obs, \
                                 self.distance_dev_rate, self.direction_dev) # Mcl's one.
        else:
            for p in self.particles: # use Mcl's motion_update().
                p.motion_update(nu, omega, \
                                self.motion_noise_rate_pdf) # Mcl's one. 